In [ ]:
import keras.layers
from keras.applications.vgg16 import VGG16, preprocess_input, decode_predictions
from keras.preprocessing.image import load_img, img_to_array
from keras import Model
import os
import numpy as np
import cv2

In [ ]:
#!jupyter nbconvert --to notebook --execute final.ipynb --output output.ipynb


In [ ]:
#from google.colab import drive
#drive.mount('/content/drive')


Mounted at /content/drive


In [ ]:
#import os
#os.chdir('/content/drive/My Drive/assia')


In [ ]:
classes = 3
batch_size = 100
epochs = 10
imageSize=224 # changing
folder_train = "/content/drive/MyDrive/assia/data/training/"
folder_val = "/content/drive/MyDrive/assia/data/validation/"

In [ ]:
import os
import cv2
X_train,y_train,X_val,y_val= [],[],[],[]
for folderName in os.listdir(folder_train):
    label=folderName[0:] # to only get the (minor-moderate-severe)
    print(label)
    for image_filename in os.listdir(folder_train + folderName): # the full path in each (moderate - ..) folders
        img = cv2.imread(folder_train + folderName + '/' + image_filename) # the full path of the image
        if img is not None:
            img = cv2.resize(img, (imageSize, imageSize))
            img_arr = np.asarray(img)
            X_train.append(img_arr)
            y_train.append(label)

for folderName in os.listdir(folder_val):
    label=folderName[0:]
    for image_filename in os.listdir(folder_val + folderName):
        img = cv2.imread(folder_val + folderName + '/' + image_filename)
        if img is not None:
            img = cv2.resize(img, (imageSize, imageSize))
            img_arr = np.asarray(img)
            X_val.append(img_arr)
            y_val.append(label)



X_train = np.asarray(X_train)
y_train = np.asarray(y_train)
X_val = np.asarray(X_val)
y_val = np.asarray(y_val)

severe
moderate
minor


In [ ]:
x_final =  np.concatenate((X_train, X_val), axis=0)
y_final = np.concatenate((y_train, y_val), axis=0)

In [ ]:
print(x_final.shape,y_final.shape)


(502, 224, 224, 3) (502,)


In [ ]:
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

In [ ]:
from sklearn.model_selection import train_test_split
import numpy as np

In [ ]:
X_train, X_test, y_train, y_test = train_test_split( x_final, y_final, test_size=0.3, random_state=42)

In [ ]:
print(X_train.shape, X_test.shape, y_train.shape, y_test.shape)

(351, 224, 224, 3) (151, 224, 224, 3) (351,) (151,)


In [ ]:
print(y_train)
# Convert labels to numerical categories
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train)
print(y_train_encoded)
y_val_encoded = label_encoder.transform(y_val)
# Convert numerical categories to one-hot encoded format
y_train_encoded = to_categorical(y_train_encoded, num_classes=3)
y_val_encoded = to_categorical(y_val_encoded, num_classes=3)

['moderate' 'severe' 'severe' 'minor' 'moderate' 'severe' 'minor' 'severe'
 'severe' 'moderate' 'minor' 'minor' 'severe' 'minor' 'minor' 'severe'
 'severe' 'minor' 'moderate' 'severe' 'moderate' 'minor' 'moderate'
 'moderate' 'minor' 'severe' 'minor' 'moderate' 'moderate' 'moderate'
 'severe' 'moderate' 'minor' 'minor' 'moderate' 'moderate' 'severe'
 'minor' 'moderate' 'minor' 'moderate' 'moderate' 'minor' 'minor'
 'moderate' 'minor' 'moderate' 'moderate' 'moderate' 'minor' 'minor'
 'severe' 'minor' 'moderate' 'moderate' 'minor' 'minor' 'severe'
 'moderate' 'moderate' 'moderate' 'moderate' 'severe' 'moderate'
 'moderate' 'moderate' 'moderate' 'moderate' 'minor' 'minor' 'moderate'
 'severe' 'severe' 'minor' 'moderate' 'moderate' 'minor' 'severe' 'severe'
 'severe' 'moderate' 'severe' 'severe' 'severe' 'moderate' 'moderate'
 'minor' 'moderate' 'moderate' 'minor' 'moderate' 'minor' 'severe'
 'severe' 'moderate' 'moderate' 'severe' 'severe' 'minor' 'moderate'
 'minor' 'severe' 'severe' 'mi

In [ ]:
y_train_encoded[:2]

array([[0., 1., 0.],
       [0., 0., 1.]], dtype=float32)

In [ ]:
model = VGG16(weights="imagenet", include_top=False, input_shape=(224, 224, 3))


In [ ]:
x = model.output
x = keras.layers.GlobalAveragePooling2D()(x)
predictions = keras.layers.Dense(classes, activation='softmax')(x)
new_model = Model(inputs=model.input, outputs=predictions)

In [ ]:
new_model.compile(loss="categorical_crossentropy", optimizer=keras.optimizers.SGD(learning_rate=0.0001, momentum=0.9), metrics=["accuracy"])


In [ ]:
for layer in model.layers[:5]:
   layer.trainable = False

In [ ]:
model_info = new_model.fit(X_train, y_train_encoded, epochs=epochs, batch_size=batch_size, verbose=2)
new_model.save("my_model.keras")


Epoch 1/10
4/4 - 709s - loss: 3.7993 - accuracy: 0.3675 - 709s/epoch - 177s/step
Epoch 2/10
4/4 - 672s - loss: 1.5665 - accuracy: 0.5299 - 672s/epoch - 168s/step
Epoch 3/10
4/4 - 671s - loss: 0.8381 - accuracy: 0.6923 - 671s/epoch - 168s/step
Epoch 4/10
4/4 - 699s - loss: 0.5908 - accuracy: 0.7521 - 699s/epoch - 175s/step
Epoch 5/10
4/4 - 672s - loss: 0.4361 - accuracy: 0.8376 - 672s/epoch - 168s/step
Epoch 6/10
4/4 - 668s - loss: 0.3386 - accuracy: 0.9060 - 668s/epoch - 167s/step
Epoch 7/10
4/4 - 667s - loss: 0.2499 - accuracy: 0.9288 - 667s/epoch - 167s/step
Epoch 8/10
4/4 - 670s - loss: 0.2027 - accuracy: 0.9402 - 670s/epoch - 167s/step
Epoch 9/10
4/4 - 667s - loss: 0.1548 - accuracy: 0.9516 - 667s/epoch - 167s/step
Epoch 10/10
4/4 - 668s - loss: 0.1201 - accuracy: 0.9658 - 668s/epoch - 167s/step


In [ ]:
predictions  = new_model.predict(X_val)
print(predictions)
# Trouver l'indice de la classe prédite pour chaque exemple
predicted_classes = np.argmax(predictions, axis=1)

print(predicted_classes)

8/8 [==============================] - 139s 17s/step
[[2.79107830e-03 1.75000932e-02 9.79708850e-01]
 [1.18122296e-02 2.78224766e-01 7.09963024e-01]
 [9.34548007e-05 1.15307383e-02 9.88375664e-01]
 [4.47664788e-04 3.10984650e-03 9.96442437e-01]
 [1.27797322e-02 2.16940522e-01 7.70279646e-01]
 [7.12444089e-05 1.26454269e-03 9.98664260e-01]
 [1.52631605e-04 2.80634668e-02 9.71783936e-01]
 [6.65496453e-04 4.85305209e-03 9.94481504e-01]
 [8.75995011e-06 2.26375926e-02 9.77353632e-01]
 [1.65790096e-02 6.34029508e-02 9.20018017e-01]
 [6.11923097e-05 1.75082143e-02 9.82430577e-01]
 [7.72519596e-03 5.75214736e-02 9.34753299e-01]
 [4.69095781e-02 5.30309156e-02 9.00059581e-01]
 [1.33055134e-03 1.07452311e-02 9.87924159e-01]
 [4.44325238e-01 2.84917891e-01 2.70756811e-01]
 [2.45813688e-04 3.02638276e-03 9.96727765e-01]
 [2.17816439e-02 4.59136143e-02 9.32304800e-01]
 [5.16291730e-05 5.60451532e-04 9.99387920e-01]
 [3.48724891e-03 4.30681445e-02 9.53444541e-01]
 [2.82814656e-03 2.41014059e-03 9.9

In [ ]:
from sklearn.metrics import accuracy_score


In [ ]:
y_val

array(['severe', 'severe', 'severe', 'severe', 'severe', 'severe',
       'severe', 'severe', 'severe', 'severe', 'severe', 'severe',
       'severe', 'severe', 'severe', 'severe', 'severe', 'severe',
       'severe', 'severe', 'severe', 'severe', 'severe', 'severe',
       'severe', 'severe', 'severe', 'severe', 'severe', 'severe',
       'severe', 'severe', 'severe', 'severe', 'severe', 'severe',
       'severe', 'severe', 'severe', 'severe', 'severe', 'severe',
       'severe', 'severe', 'severe', 'severe', 'severe', 'severe',
       'severe', 'severe', 'severe', 'severe', 'severe', 'severe',
       'severe', 'severe', 'severe', 'severe', 'severe', 'severe',
       'severe', 'severe', 'severe', 'severe', 'severe', 'severe',
       'severe', 'severe', 'severe', 'severe', 'severe', 'severe',
       'severe', 'severe', 'severe', 'severe', 'severe', 'severe',
       'severe', 'severe', 'severe', 'severe', 'severe', 'severe',
       'severe', 'severe', 'severe', 'severe', 'severe', 'seve

In [ ]:
y_encoded = label_encoder.fit_transform(y_val)
y_encoded

array([2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
       2, 2, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0])

In [ ]:
# Calculate accuracy
accuracy = accuracy_score(y_encoded, predicted_classes )

print("Accuracy:", accuracy)


Accuracy: 0.9637096774193549
